In [5]:
import warnings
import numpy as np
from numpy import cos, sin, pi
from ipyvolume import pylab as p3
import math
from ipywidgets import widgets
from ipywidgets import FloatRangeSlider, Dropdown, FloatSlider, FloatLogSlider, ColorPicker, Box, VBox, Layout, Label
from ipywidgets import interact, interactive, fixed, interact_manual

try:
    import scipy.ndimage
    import scipy.special
except:
    pass

def setup_material_widgets(surf, name, tab, index):

    if tab == None:
        return
    style = {'description_width': 'initial', 'width': 'max-content'}
    surf_color = ColorPicker(description='Color:', value=str(surf.color), continuous_update=True)
    
    surf_lighting_model = Dropdown(options=['DEFAULT', 'LAMBERT', 'PHONG', 'PHYSICAL'],value=surf.lighting_model,
                                    description='Lighting Model:', style=style)
    surf_opacity = FloatSlider(description='Opacity(Non-DEFAULT):',
                               value=surf.opacity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
    surf_specular_color = ColorPicker(description='Specular Color(Non-DEFAULT):', value=str(surf.specular_color), 
                                      continuous_update=True, style=style)
    surf_shininess = FloatSlider(description='Shininess(PHONG-Only):',
                                 value=surf.shininess, min=0.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal', 
                                readout=True, readout_format='.01f', style=style)
    surf_emissive_intensity = FloatSlider(description='Emissive intensity(Non-DEFAULT):',
                                          value=surf.emissive_intensity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal', 
                                readout=True, readout_format='.01f', style=style)
    surf_roughness = FloatSlider(description='Roughness(PHYISICAL-Only):',
                                 value=surf.roughness, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal', 
                                readout=True, readout_format='.01f', style=style)
    surf_metalness = FloatSlider(description='Metalness(PHYISICAL-Only):',
                                 value=surf.metalness, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal', 
                                readout=True, readout_format='.01f', style=style)
    
    surf_cast_shadow = widgets.Checkbox(value=True, description='Cast Shadow', style=style)
    surf_receive_shadow = widgets.Checkbox(value=True, description='Receive Shadow', style=style)
    surf_flat_shading = widgets.Checkbox(value=True, description='Flat Shading', style=style)
    
    def set_params(color, 
                   lighting_model, 
                   opacity, 
                   specular_color,
                   shininess,
                   emissive_intensity,
                   roughness,
                   metalness,
                   cast_shadow, 
                   receive_shadow,
                   flat_shading):
        surf.color = color
        surf.lighting_model = lighting_model
        surf.opacity = opacity
        surf.specular_color = specular_color
        surf.shininess = shininess
        surf.emissive_intensity = emissive_intensity
        surf.roughness = roughness
        surf.metalness = metalness
        surf.cast_shadow = cast_shadow
        surf.receive_shadow = receive_shadow
        surf.flat_shading = flat_shading
    
    interactables = interactive(set_params, 
                     color=surf_color,
                     lighting_model=surf_lighting_model, 
                     opacity=surf_opacity,
                     specular_color = surf_specular_color,
                     shininess = surf_shininess,
                     emissive_intensity = surf_emissive_intensity,
                     roughness = surf_roughness,
                     metalness = surf_metalness,
                     cast_shadow = surf_cast_shadow,
                     receive_shadow = surf_receive_shadow,
                     flat_shading = surf_flat_shading)
    
    box = VBox(children = interactables.children)
    tab.children += (box,) 
    tab.set_title(index, name)

def setup_light_widgets(light, name, tab, index):
    if tab == None:
        return
    interactables = None
    style = {'description_width': 'initial', 'width': 'max-content'}
    if light.light_type == 'AMBIENT':
        light_color = ColorPicker(description='Light Color:', value=str(light.light_color), continuous_update=True)
        light_intensity = FloatSlider(description='Intensity:',
                               value=light.intensity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
    
        def set_params_ambiental(color, 
                                intensity):
            light.light_color = color
            light.intensity = intensity
    
        interactables = interactive(set_params_ambiental, 
                                     color=light_color,
                                     intensity=light_intensity)
    elif light.light_type == 'HEMISPHERE':
        light_color = ColorPicker(description='Light Color:', value=str(light.light_color), continuous_update=True)
        light_color2 = ColorPicker(description='Light Color2:', value=str(light.light_color2), continuous_update=True)
        light_intensity = FloatSlider(description='Intensity:',
                               value=light.intensity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_x = FloatSlider(description='Position X:',
                               value=light.position_x, min=-1.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_y = FloatSlider(description='Position Y:',
                               value=light.position_y, min=-1.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_z = FloatSlider(description='Position Z:',
                               value=light.position_z, min=-1.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
    
        def set_params_hemisphere(color, color2, intensity, pos_x, pos_y, pos_z):
            light.light_color = color
            light.light_color2 = color2
            light.intensity = intensity
            light.position_x = pos_x
            light.position_y = pos_y
            light.position_z = pos_z
    
        interactables = interactive(set_params_hemisphere, 
                                    color=light_color,
                                    color2=light_color2,
                                    intensity=light_intensity,
                                    pos_x=position_x,
                                    pos_y=position_y,
                                    pos_z=position_z)
        
    elif light.light_type == 'DIRECTIONAL':
        light_color = ColorPicker(description='Light Color:', value=str(light.light_color), continuous_update=True)
        light_intensity = FloatSlider(description='Intensity:',
                               value=light.intensity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_x = FloatSlider(description='Position X:',
                               value=light.position_x, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_y = FloatSlider(description='Position Y:',
                               value=light.position_y, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_z = FloatSlider(description='Position Z:',
                               value=light.position_z, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        target_x = FloatSlider(description='Target X:',
                               value=light.target_x, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        target_y = FloatSlider(description='Target Y:',
                               value=light.target_y, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        target_z = FloatSlider(description='Target Z:',
                               value=light.target_z, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        cast_shadow = widgets.Checkbox(value=light.cast_shadow, description='Cast Shadow', style=style)
        shadow_map_size = FloatSlider(description='Shadow Map Size:',
                               value=light.shadow_map_size, min=0.0, max=1024.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_bias = FloatSlider(description='Shadow Bias:',
                               value=light.shadow_bias, min=-0.001, max=0.001, step=0.00001, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.7f', style=style)
        shadow_radius = FloatSlider(description='Shadow Radius:',
                               value=light.shadow_radius, min=0, max=10, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_near = FloatSlider(description='Shadow Camera Near:',
                               value=light.shadow_camera_near, min=0.1, max=2, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_far = FloatSlider(description='Shadow Camera Far:',
                               value=light.shadow_camera_far, min=2.01, max=5000, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_orthographic_size = FloatSlider(description='Shadow Cam Ortho Size:',
                               value=light.shadow_camera_orthographic_size, min=0, max=500, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
  
        def set_params_directional(color, intensity, pos_x, pos_y, pos_z, tar_x, tar_y, tar_z, 
                                   cast_shadow,smap_size, bias, radius, far, near, ortho_size):
            light.light_color = color
            light.intensity = intensity
            light.position_x = pos_x
            light.position_y = pos_y
            light.position_z = pos_z
            light.target_x = tar_x
            light.target_y = tar_y
            light.target_z = tar_z
            light.cast_shadow = cast_shadow
            light.shadow_map_size = smap_size
            light.shadow_bias = bias
            light.shadow_radius = radius
            light.shadow_camera_near = near
            light.shadow_camera_far = far
            light.shadow_camera_orthographic_size = ortho_size
    
        interactables = interactive(set_params_directional, 
                                    color=light_color,
                                    intensity=light_intensity,
                                    pos_x=position_x,
                                    pos_y=position_y,
                                    pos_z=position_z,
                                    tar_x=target_x,
                                    tar_y=target_y,
                                    tar_z=target_z,
                                    cast_shadow=cast_shadow,
                                    smap_size=shadow_map_size,
                                    bias=shadow_bias,
                                    radius=shadow_radius,
                                    near=shadow_camera_near,
                                    far=shadow_camera_far, 
                                    ortho_size=shadow_camera_orthographic_size)
        
    elif light.light_type == 'SPOT':
        light_color = ColorPicker(description='Light Color:', value=str(light.light_color), continuous_update=True)
        light_intensity = FloatSlider(description='Intensity:',
                               value=light.intensity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_x = FloatSlider(description='Position X:',
                               value=light.position_x, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_y = FloatSlider(description='Position Y:',
                               value=light.position_y, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_z = FloatSlider(description='Position Z:',
                               value=light.position_z, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        target_x = FloatSlider(description='Target X:',
                               value=light.target_x, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        target_y = FloatSlider(description='Target Y:',
                               value=light.target_y, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        target_z = FloatSlider(description='Target Z:',
                               value=light.target_z, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        angle = FloatSlider(description='Angle:',
                               value=light.angle, min=math.pi/100, max=math.pi/2, step=0.001, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.07f', style=style)
        penumbra = FloatSlider(description='Penumbra:',
                               value=light.penumbra, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        cast_shadow = widgets.Checkbox(value=light.cast_shadow, description='Cast Shadow', style=style)
        shadow_map_size = FloatSlider(description='Shadow Map Size:',
                               value=light.shadow_map_size, min=0.0, max=1024.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_bias = FloatSlider(description='Shadow Bias:',
                               value=light.shadow_bias, min=-0.001, max=0.001, step=0.00001, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.7f', style=style)
        shadow_radius = FloatSlider(description='Shadow Radius:',
                               value=light.shadow_radius, min=0, max=10, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_near = FloatSlider(description='Shadow Camera Near:',
                               value=light.shadow_camera_near, min=0.1, max=2, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_far = FloatSlider(description='Shadow Camera Far:',
                               value=light.shadow_camera_far, min=2.01, max=5000, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)

        def set_params_spot(color, intensity, pos_x, pos_y, pos_z, tar_x, tar_y, tar_z, 
                            angle, 
                            penumbra, 
                            cast_shadow,smap_size, bias, radius, far, near):
            light.light_color = color
            light.intensity = intensity
            light.position_x = pos_x
            light.position_y = pos_y
            light.position_z = pos_z
            light.target_x = tar_x
            light.target_y = tar_y
            light.target_z = tar_z
            light.angle = angle
            light.penumbra = penumbra
            light.cast_shadow = cast_shadow
            light.shadow_map_size = smap_size
            light.shadow_bias = bias
            light.shadow_radius = radius
            light.shadow_camera_near = near
            light.shadow_camera_far = far
    
        interactables = interactive(set_params_spot, 
                                    color=light_color,
                                    intensity=light_intensity,
                                    pos_x=position_x,
                                    pos_y=position_y,
                                    pos_z=position_z,
                                    tar_x=target_x,
                                    tar_y=target_y,
                                    tar_z=target_z,
                                    angle = angle,
                                    penumbra = penumbra,
                                    cast_shadow=cast_shadow,
                                    smap_size=shadow_map_size,
                                    bias=shadow_bias,
                                    radius=shadow_radius,
                                    near=shadow_camera_near,
                                    far=shadow_camera_far)
     
    elif light.light_type == 'POINT':
        light_color = ColorPicker(description='Light Color:', value=str(light.light_color), continuous_update=True)
        light_intensity = FloatSlider(description='Intensity:',
                               value=light.intensity, min=0.0, max=1.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_x = FloatSlider(description='Position X:',
                               value=light.position_x, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_y = FloatSlider(description='Position Y:',
                               value=light.position_y, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        position_z = FloatSlider(description='Position Z:',
                               value=light.position_z, min=-100.0, max=100.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        distance = FloatSlider(description='Max Distance:',
                               value=light.distance, min=0, max=200, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        cast_shadow = widgets.Checkbox(value=light.cast_shadow, description='Cast Shadow', style=style)
        shadow_map_size = FloatSlider(description='Shadow Map Size:',
                               value=light.shadow_map_size, min=0.0, max=1024.0, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_bias = FloatSlider(description='Shadow Bias:',
                               value=light.shadow_bias, min=-0.001, max=0.001, step=0.00001, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.7f', style=style)
        shadow_radius = FloatSlider(description='Shadow Radius:',
                               value=light.shadow_radius, min=0, max=10, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_near = FloatSlider(description='Shadow Camera Near:',
                               value=light.shadow_camera_near, min=0.1, max=2, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)
        shadow_camera_far = FloatSlider(description='Shadow Camera Far:',
                               value=light.shadow_camera_far, min=2.01, max=5000, step=0.01, 
                                continuous_update=True, orientation='horizontal',
                                readout=True, readout_format='.01f', style=style)

        def set_params_point(color, intensity, pos_x, pos_y, pos_z,  
                            distance, cast_shadow, smap_size, bias, radius, far, near):
            light.light_color = color
            light.intensity = intensity
            light.position_x = pos_x
            light.position_y = pos_y
            light.position_z = pos_z
            light.distance = distance
            light.cast_shadow = cast_shadow
            light.shadow_map_size = smap_size
            light.shadow_bias = bias
            light.shadow_radius = radius
            light.shadow_camera_near = near
            light.shadow_camera_far = far

    
        interactables = interactive(set_params_point, 
                                    color=light_color,
                                    intensity=light_intensity,
                                    pos_x=position_x,
                                    pos_y=position_y,
                                    pos_z=position_z,
                                    distance=distance,
                                    cast_shadow=cast_shadow,
                                    smap_size=shadow_map_size,
                                    bias=shadow_bias,
                                    radius=shadow_radius,
                                    near=shadow_camera_near,
                                    far=shadow_camera_far)


    if interactables:
        box = VBox(children = interactables.children)
        tab.children += (box,) 
        tab.set_title(index, name) 
    
def klein_bottle(
    endpoint=True,
    tab=None,
    index=0
):
    # http://paulbourke.net/geometry/klein/
    u = np.linspace(0, 2 * pi, num=50, endpoint=endpoint)
    v = np.linspace(0, 2 * pi, num=50, endpoint=endpoint)
    u, v = np.meshgrid(u, v)
    r = 4 * (1 - cos(u) / 2)
    x = 6 * cos(u) * (1 + sin(u)) + r * cos(u) * cos(v) * (u < pi) + r * cos(v + pi) * (u >= pi)
    y = 16 * sin(u) + r * sin(u) * cos(v) * (u < pi)
    z = r * sin(v)
    mesh = p3.plot_mesh(
                x,
                y,
                z,
                wireframe=False,
                wrapx=not endpoint,
                wrapy=not endpoint,
                u=u / (2 * np.pi),
                v=v / (2 * np.pi))
    mesh.lighting_model = 'PHYSICAL'
    setup_material_widgets(mesh, "KLEIN BOTTLE", tab, index)
    return mesh
    
def worldplane(
    tab=None,
    index=0
):
    k = 20
    h = -15
    tx = np.array([k, -k, -k, k])
    tz = np.array([k, k, -k, -k])
    ty = np.array([h, h, h, h])
    
    tri = [(0, 1, 2), (0, 2, 3)]
    p = p3.plot_trisurf(
        tx, 
        ty,
        tz, 
        triangles=tri)
    p.lighting_model = 'PHYSICAL'
    setup_material_widgets(p, "PLANE", tab, index)
    return p
    
def test_surface(
    tab=None,
    index=0):
    X = np.arange(-10, 10, 0.25*1)-10
    Y = np.arange(-10, 10, 0.25*1)
    X, Y = np.meshgrid(X, Y)
    R = np.sqrt(X**2 + Y**2)
    Z = np.sin(R)

    surf = p3.plot_surface(
        X+10, 
        Z-10, 
        Y+5)
    surf.lighting_model = 'PHYSICAL'
    setup_material_widgets(surf, "SINE SURFACE", tab, index)
    return surf

def plot_all(kb=False, ts=False, wp=True, show_ui=True):
    k=None
    w=None
    s=None
    p3.clear()
    p3.figure()
    tab = widgets.Tab() if show_ui == True else None
    index = 0

    if kb:
        k=klein_bottle(tab=tab, index=index)
        index+=1
    if ts:
        s=test_surface(tab=tab, index=index)
        index+=1
    if wp:
        w=worldplane(tab=tab, index=index)
    p3.show()
    display(tab)
    return [k,w,s] 

def lights_dir_spot():
    dir1=p3.directional_light(light_color='green', 
        intensity=1.0,
        position=[30,30,30],
        target=[0,20,30])
    sp1=p3.spot_light(light_color='yellow',
        intensity=1,
        position=[20,30,20],
        target=[-20,-20,-20],
        angle=math.pi/9,
        penumbra=0.5)

# Material properties

In [2]:
[bottle, plane, surf] = plot_all(kb=True, ts=True, wp=True)
lights_dir_spot()

# Ambient Light

In [3]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light = p3.ambient_light(light_color="green")
setup_light_widgets(light=light, name="AMBIENT LIGHT", tab=tab, index=0)
display(tab)

None

# Hemisphere Light

In [4]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light = p3.hemisphere_light()
setup_light_widgets(light=light, name="HEMISPHERE LIGHT", tab=tab, index=0)
display(tab)

None

# Directional Light

In [5]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light_d = p3.directional_light(light_color="orange")
setup_light_widgets(light=light_d, name="DIRECTIONAL LIGHT", tab=tab, index=0)
display(tab)

None

# Spot Light

In [6]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light_s = p3.spot_light(light_color="blue")
setup_light_widgets(light=light_s, name="SPOT LIGHT", tab=tab, index=0)
display(tab)

None

# Point Light

In [6]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light_p = p3.point_light(light_color="yellow")
setup_light_widgets(light=light_p, name="POINT LIGHT", tab=tab, index=0)
display(tab)

None

# Animated Light

In [36]:
[b1, p1, s1] = plot_all(kb=True, wp=True, ts=False)
point1=p3.point_light()

In [37]:
# Light and shadow Setup
point1.position_x=20
point1.position_y=10
point1.position_z=0
point1.cast_shadow = True
b1.cast_shadow = True
b1.receive_shadow = True
p1.receive_shadow = True
point1.shadow_map_type='PCF_SOFT'
point1.shadow_map_size=1024
# point1.shadow_radius=20 
point1.decay = 2

In [38]:
# Generating gif with animated light position
def set_pos(fig, i, fraction):
    point1.position_x = np.sin(2*np.pi*fraction) * 20
    point1.position_z = np.cos(2*np.pi*fraction) * 20
    
# a = p3.movie('rotatelight.gif', set_pos, fps=20, frames=40, endpoint=False, cmd_template_gif)

<img src="./rotatelight.gif" width="400" align="center">

In [ ]:
# WORK IN PROGRESS
# No need to change the lighting model manually
# whenever a light is added to the figure, the lighting model of the meshes will
# be updated to 'PHYSICAL'
#surf.lighting_model = 'PHYSICAL'
#plane.lighting_model = 'PHYSICAL'
#surf.color = 'blue'
#plane.color = 'blue'
#plane.emissive_intensity = 10
# Emissive (light) color of the material, essentially a solid color unaffected by other lighting. Default is black.

# roughness - How rough the material appears. 0.0 means a smooth mirror reflection, 1.0 means fully diffuse. 
# Default is 1.0.
# metalness - How much the material is like a metal. Non-metallic materials such as wood or stone use 0.0, 
# metallic use 1.0, with nothing (usually) in between. 
# Default is 0.0. A value between 0.0 and 1.0 could be used for a rusty metal look.
# PHYISICAL ONLY

# 1.0 - completely opaque; 0.0 - completely transparent

# specular_color - A specular highlight is the bright spot of light that appears on shiny objects when illuminated
# shininess - specular intensity
# PHONG ONLY
# Bui Tuong Phong, Illumination of Computer-Generated Images, Department of Computer Science, University of Utah, UTEC-CSs-73-129, July 1973.
# Bui Tuong Phong, "Illumination for Computer Generated Pictures," Comm. ACM, Vol 18(6):311-317, June 1975.

# Meshes can cast shadows and/or receive shadows

# Lambertian reflectance is the property that defines an ideal "matte" or diffusely reflecting surface.

#####################################################
# AMBIENT LIGHT
# light_color - default white
# intensity - default  1

#####################################################
# HEMISPHERE LIGHT
# light_color - default white - upper light
# light_color2 - default white - bottom light
# intensity - default  1
# position_x, position_y, position_z - default [0, 1, 0] 

#####################################################
# DIRECTIONAL LIGHT
# light_color - default white
# intensity - default  1
# position_x, position_y, position_z - default [0, 1, 0] 
# target_x, target_y, target_z - default [0, 0, 0]
# cast_shadow - default False
# shadow_map_size - default 512
# shadow_bias - default -0.0005
# shadow_radius - default 1
# shadow_camera_near - default 0.5
# shadow_camera_far - default 500
# shadow_camera_orthographic_size - default 100
# shadow_map_type - default 'PCF_SOFT'  - 'BASIC', 'PCF', 'PCF_SOFT'

# if some shadow settings do not update, due to a THREE.JS issue
# the settings should be updated when setting cast_shadow to False, Run cell, turn back to True and Run cell again
#directional1.cast_shadow=True
#directional1.shadow_map_size=512
#directional1.shadow_map_type='PCF_SOFT'
# Setting this to values greater than 1 will blur the edges of the shadow.
# High values will cause unwanted banding effects in the shadows - a greater mapSize will allow for a higher value to be used here before these effects become visible.
# If shadow_map_type is set to PCF_SOFT, shadow_radius has no effect and it is recommended to increase softness by decreasing mapSize instead.
#directional1.shadow_radius = 5 
#solves shadow acne issues
#directional1.shadow_bias=-0.005
#directional1.shadow_camera_orthographic_size=100
#####################################################
# POINT LIGHT
# light_color - default white
# intensity - default 1
# position_x, position_y, position_z - default [0, 1, 0]
# distance - default 0
# decay - default 1
# cast_shadow - default False
# shadow_map_size - default 512
# shadow_bias - default -0.0005
# shadow_radius - default 1
# shadow_camera_near - default 0.5
# shadow_camera_far - default 500
# shadow_map_type - default 'PCF_SOFT'  - 'BASIC', 'PCF', 'PCF_SOFT'
# point1.distance = 100 # Maximum range of the light. Default is 0 (no limit).
# # The amount the light dims along the distance of the light. 
# # Default is 1. For physically correct lighting, set this to 2.
# point1.decay = 0 
# # if some shadow settings do not update, due to a THREE.JS issue
# # the settings should be updated when setting cast_shadow to False, Run cell, turn back to True and Run cell again
# point1.cast_shadow=True
# point1.shadow_map_size=512
# point1.shadow_map_type='PCF'
# # Setting this to values greater than 1 will blur the edges of the shadow.
# # High values will cause unwanted banding effects in the shadows - a greater mapSize will allow for a higher value to be used here before these effects become visible.
# # If shadow_map_type is set to PCF_SOFT, shadow_radius has no effect and it is recommended to increase softness by decreasing mapSize instead.
# point1.shadow_radius=10 
# # solves shadow acne issues
# point1.shadow_bias=-0.005
# plane.color = 'red'
# plane.emissive_intensity = 10
# #surf.lighting_model = 'DEFAULT'
# #point1.intensity=1
# plane.color = 'green'
# plane.emissive_intensity = 100
#####################################################
# SPOT LIGHT
# light_color - default white
# intensity - default 1
# position_x, position_y, position_z - default [0, 1, 0]
# target_x, target_y, target_z - default [0, 0, 0]
# angle - default math.pi/3
# distance - default 0
# decay - default 1
# penumbra - default 0
# cast_shadow - default False
# shadow_map_size - default 512
# shadow_bias - default -0.0005
# shadow_radius - default 1
# shadow_camera_near - default 0.5
# shadow_camera_far - default 500
# shadow_camera_perspective_fov - default 50
# shadow_camera_perspective_aspect - default 1
# shadow_map_type - default 'PCF_SOFT' - 'BASIC', 'PCF', 'PCF_SOFT'

#plot_all(kb=True, wp=True, ts=False)
#spot1=p3.spot_light()

# Default mode — When distance is zero, light does not attenuate. 
# When distance is non-zero, light will attenuate linearly from maximum intensity at the 
# light's position down to zero at this distance from the light.
# Physically correct mode — When distance is zero, light will attenuate according to inverse-square law to infinite distance. 
# When distance is non-zero, light will attenuate according to inverse-square law until near the distance cutoff, 
# where it will then attenuate quickly and smoothly to 0. Inherently, cutoffs are not physically correct.
# Default is 0.0.
#spot1.distance=200

# The amount the light dims along the distance of the light.
# In physically correct mode, decay = 2 leads to physically realistic light falloff. The default is 1.
#spot1.decay=0

# Percent of the spotlight cone that is attenuated due to penumbra. 
# Takes values between zero and 1. The default is 0.0.
#spot1.penumbra=0.5

# If some shadow settings do not update, due to a THREE.JS issue
# the settings should be updated when setting cast_shadow to False, Run cell, turn back to True and Run cell again
#spot1.cast_shadow=True
#spot1.shadow_map_size=1024
#spot1.shadow_map_type='PCF_SOFT'

#spot1.shadow_camera_perspective_fov=50
# change the shadow camera aspect
#shadow_camera_perspective_aspect=1

# Setting this to values greater than 1 will blur the edges of the shadow.
# High values will cause unwanted banding effects in the shadows - a greater mapSize will allow for a higher value to be used here before these effects become visible.
# If shadow_map_type is set to PCF_SOFT, radius has no effect and it is recommended to increase softness by decreasing mapSize instead.
#spot1.shadow_radius=5 
# solves shadow acne issues
#spot1.shadow_bias=-0.0005